In [1]:
from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image

In [2]:
load_dotenv()

True

In [3]:
model = ChatMistralAI(model="mistral-large-latest")

def call_model(state: MessagesState) -> dict:
    updated_messages = model.invoke(state['messages'])
    return {"messages":updated_messages}

In [4]:
workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)
workflow.add_edge(START, "model_node")
memory = MemorySaver()
app = workflow.compile(memory)

In [5]:
chat1 = {"configurable":{"thread_id":1}}
app.invoke({"messages":"My name is Ioannis."},chat1)

{'messages': [HumanMessage(content='My name is Ioannis.', additional_kwargs={}, response_metadata={}, id='010e2182-6e35-4695-8309-cdf7eb5815df'),
  AIMessage(content='Hello Ioannis! Nice to meet you. How are you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 10, 'total_tokens': 25, 'completion_tokens': 15}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-d2a9c3db-ec22-40b7-ab57-a4febadc2d40-0', usage_metadata={'input_tokens': 10, 'output_tokens': 15, 'total_tokens': 25})]}

In [6]:
app.invoke({"messages":"What is my name?"},chat1)

{'messages': [HumanMessage(content='My name is Ioannis.', additional_kwargs={}, response_metadata={}, id='010e2182-6e35-4695-8309-cdf7eb5815df'),
  AIMessage(content='Hello Ioannis! Nice to meet you. How are you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 10, 'total_tokens': 25, 'completion_tokens': 15}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-d2a9c3db-ec22-40b7-ab57-a4febadc2d40-0', usage_metadata={'input_tokens': 10, 'output_tokens': 15, 'total_tokens': 25}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='35c32a88-732a-49cc-9f37-8a2f8fe019e1'),
  AIMessage(content='You told me your name is Ioannis.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 33, 'total_tokens': 43, 'completion_tokens': 10}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-fe471b6d-5cfc-40ad-ab8d-df282298e1a5-0', usage_metadata={'input_tokens': 33, 'output_toke

In [7]:
output = app.invoke(None, chat1)

In [8]:
[message.pretty_print() for message in output['messages']]

================================ Human Message =================================

My name is Ioannis.
================================== Ai Message ==================================

Hello Ioannis! Nice to meet you. How are you today?
================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

You told me your name is Ioannis.


[None, None, None, None]

In [9]:
chat2 = {"configurable":{"thread_id": 2}}

In [10]:
app.invoke({"messages":"What is my name?"}, chat2)['messages'][-1].content

"I'm an artificial intelligence and I don't have real-time personal information about you. Unless you've shared your name in our conversation, I wouldn't know it. If you'd like me to use a specific name when addressing you, please let me know!"

In [11]:
def chatbot(chat_id: int):

    config = {"configurable":{"thread_id": chat_id}}

    while True:
        user_input = input("User: ")

        if user_input in ["exit", "quit"]:
            print("AI: See you later!")
            break
        
        else:
           output = app.invoke({"messages":user_input}, config)
           print("AI:", output['messages'][-1].content, end="\n\n")

In [12]:
chatbot(3)

User:  Hello my name is Ioannis. How are you?


AI: Hello Ioannis! I'm functioning well, thank you. How about you? How's your day going?



User:  so far so good. 


AI: That's great to hear! What do you have planned for the rest of the day? Anything interesting?



User:  I'm planning to model data


AI: That sounds like a productive plan! What kind of data are you working with, and what tools or programming languages are you using for modeling? I might be able to provide some tips or resources if you need them. Here are a few common tools and languages for data modeling:

* Languages: Python (Pandas, NumPy, SciKit-learn), R (tidyverse, caret), SQL
* Tools: Jupyter Notebooks, RStudio, Excel, Power BI, Tableau
* Databases: MySQL, PostgreSQL, MongoDB
* Cloud Platforms: Google Cloud Platform, Amazon Web Services, Microsoft Azure



User:  exit


AI: See you later!


In [13]:
def chatbot(chat_id: int):

    config = {"configurable":{"thread_id": chat_id}}

    while True:
        user_input = input("User: ")

        if user_input in ["exit", "quit"]:
            print("AI: See you later!")
            break
        
        else:
            print("AI:", end="")
            for chunk,metadata in app.stream({"messages":user_input}, 
                               config,
                              stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")

In [14]:
chatbot(6)

User:  what is an LLM?


AI:LLM stands for **Large Language Model**. It's a type of artificial intelligence model designed to understand and generate human language based on prompts it receives. Here are some key points about LLMs:

1. **Size**: The term "large" refers to the number of parameters the model has. Parameters are what the model learns from the training data. LLMs typically have billions of parameters.

2. **Training**: LLMs are trained on vast amounts of text data from the internet. This helps them understand and generate text in a way that mimics human language.

3. **Versatility**: LLMs can perform a wide range of tasks, such as:
   - Answering questions and providing explanations.
   - Generating text, from poems to code.
   - Translating languages.
   - Summarizing text.
   - And much more.

4. **Limitations**: While LLMs can generate impressive results, they don't have personal experiences, feelings, or consciousness. They also don't have real-time information, as their knowledge cutoff is de

User:  exit


AI: See you later!
